## Packages and settings

In [1]:
import torch

from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
from sbi.utils.get_nn_models import posterior_nn
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.utils import get_density_thresholder, RestrictedPrior


/n/home11/aakhmetzhanova/.conda/envs/test_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tqdm.notebook import tqdm
from nflows import distributions as distributions_
from nflows import flows, transforms
from nflows.nn import nets

In [3]:
device = 'cpu'
print('Device: %s'%(device))

Device: cpu


## Get the simulator

In [4]:
num_dim = 2
low_priors  = torch.tensor([0.15, 0.65])
high_priors = torch.tensor([0.45, 0.95])
prior = utils.BoxUniform(low= low_priors, high=high_priors)

In [5]:
def build_maf(dim=1, num_transforms=8, context_features=None, hidden_features=128):
    transform = transforms.CompositeTransform(
        [
            transforms.CompositeTransform(
                [
                    transforms.MaskedAffineAutoregressiveTransform(
                        features=dim,
                        hidden_features=hidden_features,
                        context_features=context_features,
                        num_blocks=2,
                        use_residual_blocks=False,
                        random_mask=False,
                        activation=torch.tanh,
                        dropout_probability=0.0,
                        use_batch_norm=False,
                    ),
                    transforms.RandomPermutation(features=dim),
                ]
            )
            for _ in range(num_transforms)
        ]
    )

    distribution = distributions_.StandardNormal((dim,))
    neural_net = flows.Flow(transform, distribution)

    return neural_net


In [7]:
def flow_simulator(theta, num_samples=1):
    """ 
    Simulator by sampling from the trained normalizing flow
    """
    x = flow_net.sample(num_samples=num_samples, context=theta).detach().numpy()
    return torch.tensor(x[:, 0, :])
    

In [6]:
fmodel = ...
fout   = ...

last_layer = 16
flow_net = build_maf(dim=last_layer, context_features=2).to(device=device)
flow_net.load_state_dict(torch.load(fmodel, map_location=torch.device(device)))
flow_net.eval(); 

## Run inference

In [ ]:
# define observed summary vector s_obs; x_obs is the vector with true parameters
x_obs = torch.tensor([0.3, 0.8])
s_obs = torch.tensor([-0.0526,  0.0639,  0.1161,  0.1865,  0.2849, -0.2777,  0.0077,  0.0468,
        -0.2226,  0.0172,  0.1345, -0.1732, -0.0653,  0.1205, -0.1382, -0.1818])

flow_simulator, prior = prepare_for_sbi(flow_simulator, prior)
inference = SNPE(prior=prior)

posteriors = []
proposal = prior
num_rounds = 10

for _ in range(num_rounds):
    theta, x = simulate_for_sbi(flow_simulator, proposal, num_simulations=1000)
    density_estimator = inference.append_simulations(theta, x, proposal=proposal).train()
    
    posterior = inference.build_posterior(density_estimator)
    posteriors.append(posterior)
    proposal = posterior.set_default_x(s_obs)
    

In [ ]:
# save posteriors from each round as well as the inference summary
import os, pickle
import numpy as np 

OmegaM, sigma8 = x_obs[0], x_obs[1]
for i in range(num_rounds):
    with open(..., "wb") as handle: 
        pickle.dump(posteriors[i], handle)
        
with open(..., "wb") as handle: 
    pickle.dump(inference.summary, handle)


In [ ]:
# plot samples from posterior
posterior_samples = posterior.sample((10000,), x=s_obs)

_ = analysis.pairplot(
    posterior_samples, limits=[[0.15, 0.45], [0.65, 0.95],], figsize=(5, 5)
)